In [ ]:
from dask.distributed import Client
from dask_yarn import YarnCluster

cluster = YarnCluster(worker_memory='16GiB')
cluster.scale(4)

client = Client(cluster)

In [3]:
%load_ext autoreload
%autoreload 2
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data

In [ ]:
df = get_pvdaq_data(sysid=34, year=2011, api_key='DEMO_KEY')

In [ ]:
dh = DataHandler(df)
dh.run_pipeline(power_col='ac_power')

In [6]:
import dask

@dask.delayed
def run_pipeline(df):
    dh = DataHandler(df)
    dh.run_pipeline(power_col='ac_power')
    return dh.report(return_values=True)

In [ ]:
results = []
for _ in range(3):
    results.append(run_pipeline(df))
dask.compute(results)